In [63]:
import pandas as pd

In [64]:
# Llamar los datos de periodicidad
per_oct = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_OCT_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_nov = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_NOV_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_dic = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_DIC_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_ene = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_ENE_2023', dtype=str, usecols = ['ID_INE','Periodicidad'])

In [65]:
# Georeferenciacion de octubre 2022 a enero 2023
geo_oct = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-OCT-2022',dtype=str, usecols = ["ID_INE"]) 
geo_nov = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-NOV-2022',dtype=str, usecols = ["ID_INE"]) 
geo_dic = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-DIC-2022',dtype=str, usecols = ["ID_INE"]) 
geo_en  = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO_ENE_2023',dtype=str, usecols = ["ID_INE"]) 

In [66]:
# Fuentes asignadas de octubre 2022 a enero 2023
fuentes_oct = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-OCT-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"]) 
fuentes_nov = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-NOV-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  
fuentes_dic = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-DIC-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  
fuentes_en  = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-ENE-2023',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  

In [67]:
# Unir las bases de datos y escribe un nuevo dataframe
archivos = [per_oct,per_nov,per_dic,per_ene]
periodicidad = pd.concat(archivos,ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [68]:
# Lista de georeferenciadas
georef = [geo_oct,geo_nov,geo_dic,geo_en]

In [69]:
# Nuevo dataframe de georeferenciados 
geo_new = pd.concat(georef, ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [70]:
#Lista de fuentes asignadas 
fuentes = [fuentes_oct, fuentes_nov, fuentes_dic, fuentes_en]

In [71]:
# Agregar ID_INE en fuentes asignadas (no tenian en la primera version)
for fuente in fuentes:
    fuente.insert(7,"ID_INE",fuente.RegCod+"."+fuente.DepCod+"."+fuente.MunCod+"."+fuente.FntCod+".")

In [72]:
# Nuevo dataframe de fuentes asignadas 
fuentes_new = pd.concat(fuentes, ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [73]:
# Extraer fuentes no georeferenciadas
fuentes_new.drop(fuentes_new[fuentes_new.ID_INE.isin(geo_new.ID_INE) ].index, inplace = True)
fuentes_new = fuentes_new.reset_index()

In [74]:
# Añadir peridicidad al dataframe
fuentes_new = fuentes_new.merge(periodicidad,how = 'left', on = 'ID_INE')

In [75]:
fuentes_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6166 entries, 0 to 6165
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   level_0             6166 non-null   int64  
 1   index_x             6166 non-null   int64  
 2   RegCod              6165 non-null   object 
 3   DepCod              6166 non-null   object 
 4   MunCod              6166 non-null   object 
 5   FntCod              6166 non-null   object 
 6   NOMBRE_FUENTE       6166 non-null   object 
 7   DIRECCION           6166 non-null   object 
 8   CANTIDAD_ARTICULOS  6166 non-null   object 
 9   ID_INE              6165 non-null   object 
 10  index_y             6125 non-null   float64
 11  Periodicidad        6125 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 626.2+ KB


In [76]:
# Eliminar duplicados por nombre y dirección
fuentes_new = fuentes_new.sort_values(["Periodicidad"]).drop_duplicates(subset = ["NOMBRE_FUENTE","DIRECCION"],keep="first")

In [77]:
# Escribir nuevo documento de excel
cols = ['ID_INE','RegCod','DepCod','MunCod','FntCod','NOMBRE_FUENTE','DIRECCION','CANTIDAD_ARTICULOS','Periodicidad']
fuentes_new.to_excel('fuentes_faltan.xlsx', columns=cols, index = False)

In [78]:
fuentes_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6151 entries, 0 to 6148
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   level_0             6151 non-null   int64  
 1   index_x             6151 non-null   int64  
 2   RegCod              6150 non-null   object 
 3   DepCod              6151 non-null   object 
 4   MunCod              6151 non-null   object 
 5   FntCod              6151 non-null   object 
 6   NOMBRE_FUENTE       6151 non-null   object 
 7   DIRECCION           6151 non-null   object 
 8   CANTIDAD_ARTICULOS  6151 non-null   object 
 9   ID_INE              6150 non-null   object 
 10  index_y             6115 non-null   float64
 11  Periodicidad        6115 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 624.7+ KB
